In [2]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
# import numpy as np

# pd_jupiter = pd.read_excel('./jupyter.xlsx', engine='openpyxl')
# pd_X = pd_jupiter[['x1', 'x2', 'x3']]
# pd_Y = pd_jupiter.y
# train_X, test_X, train_Y, test_y = train_test_split(pd_X, pd_Y, test_size=0.1, random_state=42)

# treeCLF = RandomForestClassifier(random_state=0, min_samples_split=2, min_samples_leaf=1)
# parameters = {
#     'n_estimators': range(1, 5),
#     'max_depth': range(3, 5),
# }
# cv = [(slice(None), slice(None))]

# searchTree = GridSearchCV(treeCLF, parameters, cv=cv)
# searchTree.fit(train_X, train_Y)
# bestEstimatorTree = searchTree.best_estimator_
# pred_y = bestEstimatorTree.predict(test_X)
# test_y = test_y.reset_index(drop=True)
# resultDatasetPredictions = pd.DataFrame({
#     'test_y': test_y,
#     'pred_y': pred_y,
#     'diff': pred_y - test_y,
# })
# # resultDatasetPredictions
# joblib.dump(bestEstimatorTree, './jupyter/jupyter.pkl')
pd_jupiter = pd.read_csv('./jupyter.csv')
pd_jupiter.describe()

,birth_date,businessman_flag,term,contract_sum,start_date,fact_close_date,purchase_sum,purchase_count,current_balance_avg_sum,current_balance_sum,current_debit_turn_sum,current_credit_turn_sum
count,371583.000000,371583.0,0.0,3.715830e+05,318638.000000,132176.000000,3.715830e+05,371583.000000,3.715830e+05,3.715830e+05,3.715830e+05,3.715830e+05
mean,1978.405368,0.0,NaN,1.257979e+04,2019.414935,2020.601864,6.449207e+03,8.309522,2.965146e+04,2.949709e+04,2.997173e+04,2.973149e+04
std,14.195167,0.0,NaN,1.317040e+05,1.566411,1.217111,2.638060e+04,18.315616,2.194973e+05,2.219825e+05,2.331851e+05,2.384009e+05
min,1927.000000,0.0,NaN,0.000000e+00,2013.000000,2015.000000,-7.000000e+02,-1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1968.000000,0.0,NaN,0.000000e+00,2018.000000,2020.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1980.000000,0.0,NaN,0.000000e+00,2020.000000,2021.000000,0.000000e+00,0.000000,3.000000e+02,0.000000e+00,1.600000e+03,1.700000e+03
75%,1989.000000,0.0,NaN,0.000000e+00,2021.000000,2021.000000,3.700000e+03,7.000000,5.700000e+03,4.100000e+03,2.150000e+04,2.110000e+04
max,2010.000000,0.0,NaN,1.061710e+07,2022.000000,2022.000000,3.257200e+06,297.000000,1.920220e+07,1.928620e+07,5.855680e+07,5.997400e+07


In [3]:



# and later you can load it
# clf = joblib.load('filename.pkl')

['./jupiter/jupiter.pkl']

,x1,x2,x3
0,1,3,51
